# <span style="font-width:bold; font-size: 3rem; color:#333;">Training Pipeline</span>

## 🗒️ This notebook is divided into the following sections:

1. Select features for the model and create a Feature View with the selected features
2. Create training data using the feature view
3. Train model
4. Evaluate model performance
5. Save model to model registry

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import os
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
import hopsworks
from functions import util

import warnings
warnings.filterwarnings("ignore")

## <span style="color:#ff5f27;"> 📡 Connect to Hopsworks Feature Store </span>

In [2]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""

project = hopsworks.login()
fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1162392
2024-11-17 20:13:22,573 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=2,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

--- 

## <span style="color:#ff5f27;"> 🖍 Feature View Creation and Retrieving </span>

In [8]:
# Select features for training data.

selected_features = (
    air_quality_fg.select(['pm25', 'rolling_pm25'])
    .join(weather_fg.select_all(), on=['city'])
)
selected_features.show(10)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.36s) 


,pm25,rolling_pm25,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,21.0,23.000000,2021-11-11 00:00:00+00:00,14.865834,6.300000,35.045959,14.778894,barcelona
1,27.0,22.000000,2021-11-12 00:00:00+00:00,14.457501,0.800000,18.218275,312.536316,barcelona
2,34.0,24.000000,2021-11-13 00:00:00+00:00,14.728333,0.000000,12.738099,272.707855,barcelona
3,42.0,27.333334,2021-11-14 00:00:00+00:00,13.242917,4.000000,12.387348,83.946083,barcelona
4,36.0,34.333332,2021-11-15 00:00:00+00:00,12.634583,0.300000,14.058450,313.871857,barcelona
5,38.0,37.333332,2021-11-16 00:00:00+00:00,11.407501,3.000000,13.479583,304.833557,barcelona
6,38.0,38.666668,2021-11-17 00:00:00+00:00,11.203332,0.000000,16.087610,305.092468,barcelona
7,48.0,37.333332,2021-11-18 00:00:00+00:00,10.517917,0.000000,13.896187,307.342346,barcelona
8,65.0,43.000000,2021-11-20 00:00:00+00:00,10.365833,0.000000,10.315115,287.677826,barcelona
9,27.0,65.000000,2021-11-23 00:00:00+00:00,11.890834,32.099998,26.282465,70.807495,barcelona


### Feature Views

`Feature Views` are selections of features from different **Feature Groups** that make up the input and output API (or schema) for a model. A **Feature Views** can create **Training Data** and also be used in Inference to retrieve inference data.

The Feature Views allows a schema in form of a query with filters, defining a model target feature/label and additional transformation functions (declarative feature encoding).

In order to create Feature View we can use `FeatureStore.get_or_create_feature_view()` method.

You can specify the following parameters:

- `name` - name of a feature group.

- `version` - version of a feature group.

- `labels`- our target variable.

- `transformation_functions` - declarative feature encoding (not used here)

- `query` - selected features/labels for the model 

In [13]:
feature_view = fs.get_or_create_feature_view(
    name='air_quality_fv',
    description="weather features with air quality as the target",
    version=2,
    labels=['pm25'],
    query=selected_features,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1162392/fs/1153095/fv/air_quality_fv/version/2


## <span style="color:#ff5f27;">🪝 Split the training data into train/test data sets </span>

We use a time-series split here, with training data before this date `start_date_test_data` and test data after this date

In [14]:
start_date_test_data = "2024-03-01"
# Convert string to datetime object
test_start = datetime.strptime(start_date_test_data, "%Y-%m-%d")

In [15]:
read_options = {
    "arrow_flight_config": {
        "timeout": 300 
    }
}
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_start=test_start,
    read_options=read_options
)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.29s) 


In [16]:
X_train

,rolling_pm25,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,23.000000,2021-11-11 00:00:00+00:00,14.865834,6.3,35.045959,14.778894,barcelona
1,22.000000,2021-11-12 00:00:00+00:00,14.457501,0.8,18.218275,312.536316,barcelona
2,24.000000,2021-11-13 00:00:00+00:00,14.728333,0.0,12.738099,272.707855,barcelona
3,27.333334,2021-11-14 00:00:00+00:00,13.242917,4.0,12.387348,83.946083,barcelona
4,34.333332,2021-11-15 00:00:00+00:00,12.634583,0.3,14.058450,313.871857,barcelona
...,...,...,...,...,...,...,...
744,48.666668,2024-02-25 00:00:00+00:00,11.103333,0.0,22.864960,245.761642,barcelona
745,35.666668,2024-02-26 00:00:00+00:00,13.428333,0.5,23.675304,281.923157,barcelona
746,29.666666,2024-02-27 00:00:00+00:00,12.351250,0.3,15.546833,319.486267,barcelona
747,32.000000,2024-02-28 00:00:00+00:00,11.990833,0.0,19.694992,304.568359,barcelona


In [17]:
# Drop the index columns - 'date' (event_time) and 'city' (primary key)

train_features = X_train.drop(['date', 'city'], axis=1)
test_features = X_test.drop(['date', 'city'], axis=1)

In [18]:
y_train

,pm25
0,21.0
1,27.0
2,34.0
3,42.0
4,36.0
...,...
744,30.0
745,38.0
746,28.0
747,26.0


The `Feature View` is now saved in Hopsworks and you can retrieve it using `FeatureStore.get_feature_view(name='...', version=1)`.

---

## <span style="color:#ff5f27;">🧬 Modeling</span>

We will train a regression model to predict pm25 using our 4 features (wind_speed, wind_dir, temp, precipitation)

In [19]:
# Creating an instance of the XGBoost Regressor
xgb_regressor = XGBRegressor()

# Fitting the XGBoost Regressor to the training data
xgb_regressor.fit(train_features, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [20]:
# Predicting target values on the test set
y_pred = xgb_regressor.predict(test_features)

# Calculating Mean Squared Error (MSE) using sklearn
mse = mean_squared_error(y_test.iloc[:,0], y_pred)
print("MSE:", mse)

# Calculating R squared using sklearn
r2 = r2_score(y_test.iloc[:,0], y_pred)
print("R squared:", r2)

MSE: 177.14871
R squared: -0.06078568688835384


In [21]:
df = y_test
df['predicted_pm25'] = y_pred

In [23]:
df['date'] = X_test['date']
df = df.sort_values(by=['date'])
df.head(5)

,pm25,predicted_pm25,date
749,29.0,25.586821,2024-03-01 00:00:00+00:00
750,40.0,33.634964,2024-03-02 00:00:00+00:00
751,31.0,19.198664,2024-03-03 00:00:00+00:00
752,23.0,17.174404,2024-03-04 00:00:00+00:00
753,43.0,43.086601,2024-03-05 00:00:00+00:00


In [24]:
# Creating a directory for the model artifacts if it doesn't exist
model_dir = "air_quality_model"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
images_dir = model_dir + "/images"
if not os.path.exists(images_dir):
    os.mkdir(images_dir)

In [25]:
file_path = images_dir + "/pm25_hindcast.png"
plt = util.plot_air_quality_forecast("barcelona", "barcelona-carrer_de_valencia-95", df, file_path, hindcast=True) 
plt.show()

In [26]:
# Plotting feature importances using the plot_importance function from XGBoost
plot_importance(xgb_regressor, max_num_features=4)
feature_importance_path = images_dir + "/feature_importance.png"
plt.savefig(feature_importance_path)
plt.show()

---

## <span style='color:#ff5f27'>🗄 Model Registry</span>

One of the features in Hopsworks is the model registry. This is where you can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

### <span style="color:#ff5f27;">⚙️ Model Schema</span>

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/machine-learning-api/latest/generated/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [27]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Creating input and output schemas using the 'Schema' class for features (X) and target variable (y)
input_schema = Schema(X_train)
output_schema = Schema(y_train)

# Creating a model schema using 'ModelSchema' with the input and output schemas
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

# Converting the model schema to a dictionary representation
schema_dict = model_schema.to_dict()

In [28]:
# Saving the XGBoost regressor object as a json file in the model directory
xgb_regressor.save_model(model_dir + "/model.json")

In [29]:
res_dict = { 
        "MSE": str(mse),
        "R squared": str(r2),
    }

In [30]:
mr = project.get_model_registry()

# Creating a Python model in the model registry named 'air_quality_xgboost_model'

aq_model = mr.python.create_model(
    name="air_quality_xgboost_model", 
    metrics= res_dict,
    model_schema=model_schema,
    input_example=X_test.sample().values, 
    description="Air Quality (PM2.5) predictor",
)

# Saving the model artifacts to the 'air_quality_model' directory in the model registry
aq_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/421854 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/19393 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/112046 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/126 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/720 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1162392/models/air_quality_xgboost_model/3


Model(name: 'air_quality_xgboost_model', version: 3)

---
## <span style="color:#ff5f27;">⏭️ **Next:** Part 04: Batch Inference</span>

In the following notebook you will use your model for Batch Inference.
